In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

**Loading images**

In [5]:
new_base_dir = Path("cats_vs_dogs_small")

In [6]:
train_dataset = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_dataset = image_dataset_from_directory(new_base_dir / "validation", image_size=(180, 180), batch_size=32)
test_dataset = image_dataset_from_directory(new_base_dir / "test", image_size=(180, 180), batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


**Printing the list of trainable weights before and after freezing**

In [17]:
vgg16 = VGG16(weights="imagenet", include_top=False)

In [18]:
vgg16.trainable = True
print("Number of trainable weights before freezing the VGG16:", len(vgg16.trainable_weights))

Number of trainable weights before freezing the VGG16: 26


In [19]:
vgg16.trainable = False
print("Number of trainable weights after freezing the VGG16:", len(vgg16.trainable_weights))

Number of trainable weights after freezing the VGG16: 0


**Data Augmentation**

In [20]:
augmentation = Sequential([RandomFlip("horizontal"), RandomRotation(0.1),RandomZoom(0.2)])

In [21]:
inputs = Input(shape=(180, 180, 3))

x = augmentation(inputs)

x = preprocess_input(x)

x = vgg16(x)
x = Flatten()(x)
x = Dense(256)(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [22]:
callbacks = [ModelCheckpoint(filepath="feature_extraction_augmented.keras", save_best_only=True, monitor="val_loss")]
history = model.fit(train_dataset, epochs=50, validation_data=validation_dataset, callbacks=callbacks)

Epoch 1/50
63/63 [==============================] - 16s 220ms/step - loss: 18.6574 - accuracy: 0.9040 - val_loss: 15.1693 - val_accuracy: 0.9220
Epoch 2/50
63/63 [==============================] - 14s 225ms/step - loss: 7.5288 - accuracy: 0.9445 - val_loss: 9.9590 - val_accuracy: 0.9470
Epoch 3/50
63/63 [==============================] - 14s 215ms/step - loss: 4.4232 - accuracy: 0.9670 - val_loss: 4.6945 - val_accuracy: 0.9700
Epoch 4/50
63/63 [==============================] - 14s 214ms/step - loss: 4.6150 - accuracy: 0.9620 - val_loss: 4.0974 - val_accuracy: 0.9730
Epoch 5/50
63/63 [==============================] - 14s 214ms/step - loss: 3.5417 - accuracy: 0.9700 - val_loss: 3.9712 - val_accuracy: 0.9690
Epoch 6/50
63/63 [==============================] - 13s 214ms/step - loss: 3.0648 - accuracy: 0.9730 - val_loss: 2.4376 - val_accuracy: 0.9790
Epoch 7/50
63/63 [==============================] - 13s 212ms/step - loss: 2.6629 - accuracy: 0.9790 - val_loss: 3.8252 - val_accuracy: 0.97

63/63 [==============================] - 14s 219ms/step - loss: 0.6054 - accuracy: 0.9905 - val_loss: 4.3785 - val_accuracy: 0.9710
Epoch 42/50
63/63 [==============================] - 13s 213ms/step - loss: 0.3923 - accuracy: 0.9945 - val_loss: 4.5647 - val_accuracy: 0.9770
Epoch 43/50
63/63 [==============================] - 14s 215ms/step - loss: 0.2729 - accuracy: 0.9955 - val_loss: 4.8092 - val_accuracy: 0.9690
Epoch 44/50
63/63 [==============================] - 13s 213ms/step - loss: 0.6759 - accuracy: 0.9895 - val_loss: 3.6997 - val_accuracy: 0.9730
Epoch 45/50
63/63 [==============================] - 14s 217ms/step - loss: 0.6693 - accuracy: 0.9875 - val_loss: 3.5299 - val_accuracy: 0.9750
Epoch 46/50
63/63 [==============================] - 13s 213ms/step - loss: 0.2871 - accuracy: 0.9955 - val_loss: 4.4750 - val_accuracy: 0.9740
Epoch 47/50
63/63 [==============================] - 14s 215ms/step - loss: 0.6257 - accuracy: 0.9900 - val_loss: 5.4966 - val_accuracy: 0.9690
Epoc

**Evaluating the model on the test set**

In [24]:
test_model = load_model("feature_extraction_augmented.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 5s 71ms/step - loss: 5.7719 - accuracy: 0.9700
Test accuracy: 0.970
